In [1]:
## imports
import os 

import numpy as np
import cv2 as cv

%matplotlib nbagg
# %matplotlib inline
import matplotlib.pyplot as plt

from skimage import data, img_as_float
from skimage.segmentation import (morphological_chan_vese, checkerboard_level_set)
from scipy.stats import skew

import bz2 # to zip the pickle file: 
import pickle

%reload_ext autoreload
%autoreload 2
%aimport OCT_lib

In [12]:
initial_path = r"C:\Users\user\Google Drive\01 - OCT\Measurements\20190718_PDMS_Sample20180219_2\\" 

# folder_name = os.path.basename(os.path.normpath(initial_path))

## unpickling
# VU U-021
# pckl = r"C:\Users\user\Google Drive\01 - OCT\Measurements\20190718_PDMS_Sample20180219_2\processed\20190718_PDMS_Sample20180219_2_processed.bz2"
# Home Desktop
# pckl = r"C:\Users\lucab\Google Drive\01 - OCT\Measurements\20190718_PDMS_Sample20180219_2\processed\20190718_PDMS_Sample20180219_2_processed.bz2"

#############
# pckl = OCT_lib.get_file(initdir = initial_path, ext='bz2')

# init_folder = os.path.dirname(pckl)

# sfile = bz2.BZ2File(pckl, 'r')
# data = pickle.loads(sfile.read())
# sfile.close()

# print(f"Dataset: \n{folder_name}\nsuccessfully read")
#####
initial_path = r"/Users/lucab/Google Drive/01 - OCT/Measurements/20190410 - step PDMS 2\processed\\" 
data, basepath = OCT_lib.get_results(initialdir=initial_path)
folder_name = data.pop('name')
p = data.pop('params') # p is the dict with all previous analysis settings
data = data.pop('results')
experiments = list(data.keys()) 

Dataset: 
C:/Users/lucab/Google Drive/01 - OCT/Measurements/20190410 - step PDMS 2
successfully read


In [ ]:
# for this particular set of measurements on PDMS (20190718_PDMS_Sample20180219_2)
regions = ['thin', 'thick', 'boundary']
# repeated measurements
reps = ['001', '002', '003', '004', '005']

N_locs = len(regions) # locations

In [ ]:
## Conversions px to real distance

# aperture size is p['OCT']['aperture_size'], divided by the number of columns (A-scans) that it takes to image it
# assuming the data has been acquired at the maximum width of the aperture
px_to_d_hor = p['OCT']['aperture_size']/len(data[experiments[0]]["profile"]) 
hor = np.squeeze(np.arange(0,p['OCT']['aperture_size'],px_to_d_hor))

# the axial resolution of the OCT is 5.75 um/px for an ascanlength of 1024 elements
# The following expression also considers the case in which the image has been resized
px_to_d_vert = p['OCT']['axial_res']/(p['resize']['height_factor'] if p['resize']['switch'] else 1)

In [ ]:
# `deltaZ` blueprint -> profile[region][rep]
deltaZ = {}
for i, meas in enumerate(data.keys()): # iterate over all meeasurements 
    for region in regions: # iterate over the three regions
        deltaZ[region]=[]
        for rep in reps: # iterate over the repeated 
            name_rel = region+'_rel_'+rep
            name_suc = f"{region}_suc_{rep}"
            deltaZ[region].append(
                np.squeeze(np.array(
                    ((data[name_suc]["profile"])-(data[name_suc]["profile"][0])) -
                    ((data[name_rel]["profile"])-(data[name_rel]["profile"][0]))                    
                )*-px_to_d_vert)
            )

In [ ]:
# used for Asymmetry Factor calculations
peak_cut = 0.2

for region in regions:
    for i,rep in enumerate(reps):
        name_rel = f"{region}_rel_{rep}"
        name_suc = f"{region}_suc_{rep}"
        height, width = data[name_rel]['image'].shape
        
        delta_z = deltaZ[region][i]
        delta_z_sm = OCT_lib.smooth_profile(delta_z)
        x_range = OCT_lib.center_profile(hor, delta_z_sm)
        
        peak_z = max(delta_z_sm)
        peak_idx = int(np.argwhere(delta_z_sm == peak_z)[0])
        peak_left_idx  = int(np.argwhere(delta_z_sm > peak_z*peak_cut)[0]) 
        peak_right_idx = int(np.argwhere(delta_z_sm > peak_z*peak_cut)[-1]) 
        peak_base_middle_idx = int((peak_right_idx+peak_left_idx)/2) #used for AF3
        
        fig, ax = plt.subplots(nrows=2, ncols=2)

        ax[0,0].imshow(data[name_rel]['image'], cmap='gray', extent=[0, width, 0, height])
        ax[0,0].axis('off')
        ax[0,0].plot(height-data[name_rel]['profile'], color='red', alpha=0.5)
        ax[0,0].scatter(peak_x, height-data[name_rel]['profile'][peak_x], s = 7, c='forestgreen', alpha=0.7, zorder = 200 )
        ax[0,0].set_title(f"Relaxed")
        
        ax[0,1].imshow(data[name_suc]['image'], cmap='gray', extent=[0, width, 0, height])
        ax[0,1].axis('off')
        ax[0,1].plot(height-data[name_suc]['profile'], color='red', alpha=0.5)
        ax[0,1].scatter(peak_x, height-data[name_suc]['profile'][peak_x], s = 7, c='forestgreen', alpha=0.7, zorder = 200 ) 
        ax[0,1].set_title(f"Suction = 500mbar")
        
        ax[1,0].plot(x_range, delta_z, alpha=0.5, c = 'midnightblue', label = "profile")
        ax[1,0].plot(x_range, delta_z_sm, zorder =100, c = 'red', linewidth = 1.7, label="smoothed")
        ax[1,0].set_xlim(left = -p['OCT']['aperture_size']/1.85, right = p['OCT']['aperture_size']/1.85)
#         ax[1,0].axvline(x=0, c='grey', alpha =0.5)
#         ax[1,0].axhline(y = max(delta_z_sm)*peak_cut, c='grey', alpha = 0.5)

        ax[1,0].fill_between(
            ## AF2
#             x  = x_range[np.logical_and(x_range>0, delta_z_sm > max(delta_z_sm)*peak_cut )], 
#             y2 = delta_z_sm[np.logical_and(x_range>0, delta_z_sm > max(delta_z_sm)*peak_cut)],
            ## AF3
            x = x_range[peak_base_middle_idx:peak_right_idx], 
            y2 = delta_z_sm[peak_base_middle_idx:peak_right_idx], 
            ## common
            y1 = max(delta_z_sm)*peak_cut,
            facecolor = "forestgreen",
            alpha = 0.3)
        ax[1,0].fill_between(
            ## AF2            
#             x  = x_range[np.logical_and(x_range<0, delta_z_sm > max(delta_z_sm)*peak_cut )], 
#             y2 = delta_z_sm[np.logical_and(x_range<0, delta_z_sm > max(delta_z_sm)*peak_cut)],
            ## AF3
            x = x_range[peak_left_idx:peak_base_middle_idx], 
            y2 = delta_z_sm[peak_left_idx:peak_base_middle_idx],
            ## common
            y1 = max(delta_z_sm)*peak_cut,
            facecolor = "red",
            alpha = 0.3)
        
        ax[1,0].set(ylabel='$\Delta$z ($\mu$m)')
        ax[1,0].set(xlabel='Position with respect to the peak (mm)')
#         ax[1,0].legend()
        ax[1,0].spines['bottom'].set_smart_bounds(True)
        ax[1,0].spines['left'].set_smart_bounds(True)
        ax[1,0].spines['top'].set_color('none')
        ax[1,0].spines['right'].set_color('none')
#         ax[1,0].spines['bottom'].set_position(('axes', -0.05))
        
        af1, af2, af3 = OCT_lib.asymmetry_factor(delta_z_sm, peak_cutoff = peak_cut)
        info = ''\
        f"Max Delta Z = {max(delta_z_sm):.4}\n"\
        f"Asymm. factor 1 = {af1:.2}\n"\
        f"Asymm. factor 2 = {af2:.2}\n"\
        f"Asymm. factor 3 = {af3:.2}\n"\
        f"Sign(af2) * af1 = {np.sign(af2)*af1:.3}\n"\
        f"Sign(af3) * af1 = {np.sign(af3)*af1:.3}\n"
#         f"Skewness = {skew(delta_z_sm):.4}\n"
        ax[1,1].axis('off')
        ax[1,1].annotate(info,(0.1, 0.5),xycoords='axes fraction', va='center')
        
        fig.tight_layout()
        
        sup_title = f"B-scans and deformation measurement on '{region}-{rep}'"
        fig.suptitle(sup_title, fontsize=14)
        fig.subplots_adjust(top=0.93)
        
        figname = f"{init_folder}\\{sup_title}_CV.png"
        fig.savefig(fname = figname, dpi = 450)
        plt.close(fig)        
        
#         break
#     break
